In [34]:
import pandas as pd
import requests
import json
import glob, os
import numpy as np
api = 'c4ad87b03bec1878bc0be6156d4472b0'
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',100)
pd.set_option('display.min_rows',20)

In [10]:
ticker = 'AAPL'
ticker_list = requests.get(f'https://financialmodelingprep.com/api/v3/available-traded/list?apikey={api}').json()
profile = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api}').json()

In [9]:
ticker_df = pd.DataFrame(ticker_list)
ticker_df.head()

,symbol,name,price,exchange,exchangeShortName
0,SPY,SPDR S&P 500 ETF Trust,411.99,New York Stock Exchange Arca,AMEX
1,CMCSA,Comcast Corporation,37.52,NASDAQ Global Select,NASDAQ
2,KMI,"Kinder Morgan, Inc.",17.99,New York Stock Exchange,NYSE
3,INTC,Intel Corporation,36.31,NASDAQ Global Select,NASDAQ
4,MU,"Micron Technology, Inc.",61.86,NASDAQ Global Select,NASDAQ


In [11]:
profile

[{'symbol': 'AAPL',
  'price': 162.51,
  'beta': 1.19455,
  'volAvg': 90071827,
  'mktCap': 2639649832960,
  'lastDiv': 0.89,
  'range': '129.04-182.94',
  'changes': 5.1599884,
  'companyName': 'Apple Inc.',
  'currency': 'USD',
  'cik': '0000320193',
  'isin': 'US0378331005',
  'cusip': '037833100',
  'exchange': 'NASDAQ Global Select',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Consumer Electronics',
  'website': 'https://www.apple.com',
  'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services

In [16]:
def get_profile(ticker):
    profile_request = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api}').json()
    

    #dates_df = pd.DataFrame.from_dict(prices_request,orient='columns')
    out_file = open(f"{ticker}.json", "w") 
    json.dump(profile_request, out_file, indent = 4) 
    out_file.close() 
    
    with open(f'{ticker}.json','r') as f:
        data = json.loads(f.read())
    # Flatten data
    df_nested_list = pd.json_normalize(data)
    df_nested_list['Ticker'] = ticker
    df_nested_list.to_csv(f"profile/{ticker}_profile.csv")
    #return df_nested_list


In [17]:
get_profile('AAPL')

In [25]:
SP500_df = pd.read_csv('sp500_constituent.csv')
tickers = SP500_df['symbol']

result = [get_profile(ticker) for ticker in tickers]

In [31]:
path = '/Users/brendan/Desktop/Python/BoostedAI/profile'
csv_files = [f for f in os.listdir(path) if f.endswith('csv')]
os.chdir('/Users/brendan/Desktop/Python/BoostedAI/profile')
os.getcwd()

csv_to_df = [pd.DataFrame(pd.read_csv(file)) for file in csv_files]

raw_data_df = []
for file in csv_files:
    file = pd.read_csv(file)
    df = pd.DataFrame(file)
    #df['ticker'] = file
    raw_data_df.append(df)
master_df = pd.concat(raw_data_df)


In [40]:
os.chdir('/Users/brendan/Desktop/Python/BoostedAI')
os.getcwd()
master_df.to_csv('master_df.csv')

In [36]:
master_df.head()

,Unnamed: 0,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,cik,isin,cusip,exchange,exchangeShortName,industry,website,description,ceo,sector,country,fullTimeEmployees,phone,address,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund,Ticker
0,0,MOS,52.66,1.65,"7,543,867.00","19,062,550,528.00",0.41,29.14-79.28,1.86,The Mosaic Company,USD,"1,285,785.00",US61945C1036,61945C103,New York Stock Exchange,NYSE,Agricultural Inputs,https://www.mosaicco.com,"The Mosaic Company, through its subsidiaries, produces and markets concentrated phosphate and potash crop nutrients in North America and internationally. The company operates through three segments: Phosphates, Potash, and Mosaic Fertilizantes. It owns and operates mines, which produce concentrated phosphate crop nutrients, such as diammonium phosphate, monoammonium phosphate, and ammoniated phosphate products; and phosphate-based animal feed ingredients primarily under the Biofos and Nexfos brand names, as well as produces a double sulfate of potash magnesia product under K-Mag brand name. The company also produces and sells potash for use in the manufacturing of mixed crop nutrients and animal feed ingredients, and for industrial use; and for use in the de-icing and as a water softener regenerant. In addition, it provides nitrogen-based crop nutrients, animal feed ingredients, and other ancillary services; and purchases and sells phosphates, potash, and nitrogen products. The company sells its products to wholesale distributors, retail chains, farmers, cooperatives, independent retailers, and national accounts. The Mosaic Company was incorporated in 2004 and is headquartered in Tampa, Florida.",Mr. James O'Rourke,Basic Materials,US,"12,525.00",18137754200,101 E Kennedy Blvd Ste 2500,Tampa,FLORIDA,33602,2.02,52.82,https://financialmodelingprep.com/image-stock/MOS.png,2011-05-20,False,False,True,False,False,MOS
0,0,GRMN,97.62,1.01,"1,002,119.00","19,728,025,600.00",2.74,92.31-178.81,-0.42,Garmin Ltd.,USD,"1,121,788.00",CH0114405324,H2906T109,New York Stock Exchange,NYSE,Scientific & Technical Instruments,https://www.garmin.com,"Garmin Ltd. designs, develops, manufactures, markets, and distributes a range of wireless devices in the Americas, the Asia Pacific, Australian Continent, Europe, the Middle East, and Africa. Its Fitness segment offers running and multi-sport watches; cycling products; activity tracking and smartwatch devices; and fitness and cycling accessories. This segment also provides Garmin Connect and Garmin Connect Mobile, which are web and mobile platforms; and Connect IQ, an application development platform. The company's Outdoor segment offers adventure watches, outdoor handhelds, golf devices and mobile apps, and dog tracking and training devices. Its Aviation segment designs, manufactures, and markets various aircraft avionics solutions comprising integrated flight decks, electronic flight displays and instrumentation, navigation and communication products, automatic flight control systems and safety-enhancing technologies, audio control systems, engine indication systems, traffic awareness and avoidance solutions, ADS-B and transponder solutions, weather information and avoidance solutions, datalink and connectivity solutions, portable GPS navigators and wearables, and various services products. The company's Marine segment provides chartplotters and multi-function displays, cartography products, fish finders, sonar products, autopilot systems, radars, compliant instrument displays and sensors, VHF communication radios, handhelds and wearable devices, sailing products, entertainment, digital switching products, and trolling motors. Its Auto segment offers embedded computing models and infotainment systems; personal navigation devices; and cameras. The company sells its products through independent retailers, online retailers, dealers, distributors, installation and repair shops, and original equip

In [50]:
def df_sector():
    df = pd.read_csv('/Users/brendan/Desktop/Python/BoostedAI/master_df.csv')
    df = df[['symbol','mktCap', 'price','companyName','exchangeShortName','sector','industry']]
    return df

In [51]:
df_sector().head()

,symbol,mktCap,price,companyName,exchangeShortName,sector,industry
0,MOS,"19,062,550,528.00",52.66,The Mosaic Company,NYSE,Basic Materials,Agricultural Inputs
1,GRMN,"19,728,025,600.00",97.62,Garmin Ltd.,NYSE,Technology,Scientific & Technical Instruments
2,DGX,"16,029,221,888.00",136.57,Quest Diagnostics Incorporated,NYSE,Healthcare,Diagnostics & Research
3,ATO,"16,875,031,552.00",121.39,Atmos Energy Corporation,NYSE,Utilities,Utilities—Regulated Gas
4,IQV,"44,812,279,808.00",240.27,IQVIA Holdings Inc.,NYSE,Healthcare,Diagnostics & Research
